# Word Window Classifier

The aim of this notebook is to define a NLP neural network able to detect the color-related words contained in a sentence.\
Inspired by [Standford's CS224N lab](https://stanford.edu/class/cs224n/materials/CS224N_PyTorch_Tutorial.html).

## Modules Importation

In [24]:
import torch
import torch.nn as nn

from typing import List, Tuple, Dict
from torch.utils.data import DataLoader
from functools import partial

## Corpus Definition

The present dataset is composed by different sentences.\
The purpose of the model is to be able to recognize a color among these words.

In [25]:
sentences = [
    "The sky was a brilliant shade of blue.",
    "She wore a stunning red dress to the party.",
    "The leaves turned vibrant shades of orange and yellow in the fall.",
    "His new car is a sleek black.",
    "The walls of the room were painted a calming light green.",
    "A fluffy white cat sat on the windowsill.",
    "The sunset was a beautiful mix of pink and purple.",
    "She admired the emerald green of the gemstone.",
    "The artist used a lot of bright yellow in his painting.",
    "The ocean looked turquoise under the midday sun.",
    "He bought a pair of brown leather shoes.",
    "The morning sky was a soft pastel pink.",
    "The old book had a faded blue cover.",
    "Her eyes were a striking shade of hazel.",
    "The banana was ripe and yellow.",
    "The house had a cheerful red front door.",
    "The mountain peaks were capped with white snow.",
    "She chose a light grey suit for the interview.",
    "The grapes were a dark purple, almost black.",
    "The sunset painted the horizon with orange and red hues.",
    "The bird had bright blue feathers.",
    "The grass was lush and green after the rain.",
    "He preferred writing with a black pen.",
    "Her hair was dyed a vivid purple.",
    "The butterfly had wings of brilliant blue.",
    "The apples were shiny and red.",
    "The night sky was a deep, inky black.",
    "She decorated the room with light lavender curtains.",
    "Who are these greens snakes that are whistling on your heads for?"
]


## Data Preprocessing

Usually, when starting a NLP task, it is recommanded to preprocess the given sentences this way:\
\- lowering each character;\
\- splitting the sentence into tokens (i.e. an array where each element is a word);\
\- removing stop words;\
\- stemming or lemmatizing the words.

With regards to the fact that this notebook is mainly made to expose word window classifiers, the two last steps are not going to be performed as they are not required.

In [26]:
punctuation = set([',', ';', '.', '?', '!', '/', "'", '-', '_'])

def tokenize(sentence: str) -> List[str]:
    for punc in punctuation: sentence = sentence.replace(punc, '')
    return sentence.lower().split(' ')

train_sentences = [tokenize(sentence) for sentence in sentences]

# Example 
train_sentences[0]

['the', 'sky', 'was', 'a', 'brilliant', 'shade', 'of', 'blue']

The input sentence `"The sky was a brilliant shade of blue."` is transfored into the array of tokens `['the', 'sky', 'was', 'a', 'brilliant', 'shade', 'of', 'blue']`, which is actually better when it comes to machine learning tasks.

The following array defines the labels of the input words. For sentence, the array contains the name number of integers as the input array has words. These integers are equal to `1`if the word refers to a color, else `0`.

In [27]:
train_labels = [
    [0, 0, 0, 0, 0, 0, 0, 1],              # "The sky was a brilliant shade of blue."
    [0, 0, 0, 0, 1, 0, 0, 0, 0],           # "She wore a stunning red dress to the party."
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],  # "The leaves turned vibrant shades of orange and yellow in the fall."
    [0, 0, 0, 0, 0, 0, 1],                 # "His new car is a sleek black."
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],     # "The walls of the room were painted a calming light green."
    [0, 0, 1, 0, 0, 0, 0, 0],              # "A fluffy white cat sat on the windowsill."
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 1],        # "The sunset was a beautiful mix of pink and purple."
    [0, 0, 0, 0, 1, 0, 0, 0],              # "She admired the emerald green of the gemstone."
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],     # "The artist used a lot of bright yellow in his painting."
    [0, 0, 0, 1, 0, 0, 0, 0],              # "The ocean looked turquoise under the midday sun."
    [0, 0, 0, 0, 0, 1, 0, 0],              # "He bought a pair of brown leather shoes."
    [0, 0, 0, 0, 0, 0, 0, 1],              # "The morning sky was a soft pastel pink."
    [0, 0, 0, 0, 0, 0, 1, 0],              # "The old book had a faded blue cover."
    [0, 0, 0, 0, 0, 0, 0, 1],              # "Her eyes were a striking shade of hazel."
    [0, 0, 0, 0, 0, 1],                    # "The banana was ripe and yellow."
    [0, 0, 0, 0, 0, 1, 0, 0],              # "The house had a cheerful red front door."
    [0, 0, 0, 0, 0, 0, 1, 0],              # "The mountain peaks were capped with white snow."
    [0, 0, 0, 0, 1, 0, 0, 0, 0],           # "She chose a light grey suit for the interview."
    [0, 0, 0, 0, 0, 1, 0, 1],              # "The grapes were a dark purple, almost black."
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0],        # "The sunset painted the horizon with orange and red hues."
    [0, 0, 0, 0, 1, 0],                    # "The bird had bright blue feathers."
    [0, 0, 0, 0, 0, 1, 0, 0, 0],           # "The grass was lush and green after the rain."
    [0, 0, 0, 0, 0, 1, 0],                 # "He preferred writing with a black pen."
    [0, 0, 0, 0, 0, 0, 1],                 # "Her hair was dyed a vivid purple."
    [0, 0, 0, 0, 0, 0, 1],                 # "The butterfly had wings of brilliant blue."
    [0, 0, 0, 0, 0, 1],                    # "The apples were shiny and red."
    [0, 0, 0, 0, 0, 0, 0, 1],              # "The night sky was a deep, inky black."
    [0, 0, 0, 0, 0, 0, 1, 0],              # "She decorated the room with light lavender curtains."
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]   # "Who are these greens snakes that are whistling on your heads for?"
]


It is convenient for the following tasks to define a set of words, here called `vocabulary`.\
This set contains every word contained in the dataset's sentences.

In [28]:
vocabulary = set(word for sentence in train_sentences for word in sentence)
vocabulary

{'a',
 'admired',
 'after',
 'almost',
 'and',
 'apples',
 'are',
 'artist',
 'banana',
 'beautiful',
 'bird',
 'black',
 'blue',
 'book',
 'bought',
 'bright',
 'brilliant',
 'brown',
 'butterfly',
 'calming',
 'capped',
 'car',
 'cat',
 'cheerful',
 'chose',
 'cover',
 'curtains',
 'dark',
 'decorated',
 'deep',
 'door',
 'dress',
 'dyed',
 'emerald',
 'eyes',
 'faded',
 'fall',
 'feathers',
 'fluffy',
 'for',
 'front',
 'gemstone',
 'grapes',
 'grass',
 'green',
 'greens',
 'grey',
 'had',
 'hair',
 'hazel',
 'he',
 'heads',
 'her',
 'his',
 'horizon',
 'house',
 'hues',
 'in',
 'inky',
 'interview',
 'is',
 'lavender',
 'leather',
 'leaves',
 'light',
 'looked',
 'lot',
 'lush',
 'midday',
 'mix',
 'morning',
 'mountain',
 'new',
 'night',
 'ocean',
 'of',
 'old',
 'on',
 'orange',
 'painted',
 'painting',
 'pair',
 'party',
 'pastel',
 'peaks',
 'pen',
 'pink',
 'preferred',
 'purple',
 'rain',
 'red',
 'ripe',
 'room',
 'sat',
 'shade',
 'shades',
 'she',
 'shiny',
 'shoes',
 'sk

When asking the model's to predict what is the color(s) of a sentence, it is possible that it faces a word that is not in its vocabulary.\
To prevent from this, the unknown word is added to the set of words as `'<unk>'`.\
Also, as the models relies on a window of words sliding through the sentence, it is required to define another special words to handle the side effects. This padding word is defined as `'<pad>'`.

In [29]:
vocabulary.add('<unk>')
vocabulary.add('<pad>')

In [30]:
def pad_window(sentence: List[str], window_size: int, pad_token='<pad>') -> List[str]:
    window = [pad_token] * window_size
    return window + sentence + window

# Example
print(f'Padded sentence: {pad_window(train_sentences[0], window_size=2)}')

Padded sentence: ['<pad>', '<pad>', 'the', 'sky', 'was', 'a', 'brilliant', 'shade', 'of', 'blue', '<pad>', '<pad>']


It is now possible to automatically add padding to each tokenized sentences.\
The next step to make this dataset compatible with machine learning models is to give to each word a unique index it can refer to.

In [31]:
word_to_index = {word: index for index, word in enumerate(sorted(list(vocabulary)))}
index_to_word = {index: word for index, word in enumerate(sorted(list(vocabulary)))}

Two functions are now defined to process this way the entire dataset.

In [32]:
def tokens_to_indices(sentence: List[str], word_to_index: Dict[str, int]) -> List[int]:
    return [word_to_index[token] if token in word_to_index else word_to_index['<unk>'] for token in sentence]

def indices_to_token(indices: List[int], index_to_word: Dict[int, str]) -> List[str]:
    return [index_to_word[index] for index in indices]

sentence_ver = 'Are birds red?'
tokenized_ver = tokenize(sentence_ver)
indices_ver = tokens_to_indices(tokenized_ver, word_to_index)
restored_ver = indices_to_token(indices_ver, index_to_word)

print(f'Original: {sentence_ver}')
print(f'Tokenized: {tokenized_ver}')
print(f'Indices: {indices_ver}')
print(f'Restored: {restored_ver}')

Original: Are birds red?
Tokenized: ['are', 'birds', 'red']
Indices: [8, 1, 92]
Restored: ['are', '<unk>', 'red']


As explained before, "birds" is not a word of the set of words previously defined. Thus, is it replaced by `'<unk>'`.

In [33]:
train_indices = [tokens_to_indices(sentence, word_to_index) for sentence in train_sentences]
train_indices[:5]

[[112, 101, 122, 2, 18, 96, 77, 14],
 [98, 130, 2, 107, 92, 33, 114, 112, 84],
 [112, 65, 115, 119, 97, 77, 80, 6, 132, 59, 112, 38],
 [55, 74, 23, 62, 2, 102, 13],
 [112, 121, 77, 112, 94, 123, 81, 2, 21, 66, 46]]

## Model Definition

In [34]:
embeds = nn.Embedding(len(vocabulary), embedding_dim=5)
list(embeds.parameters())

[Parameter containing:
 tensor([[ 1.4400e+00, -2.6946e-01,  6.6482e-01,  1.1632e+00, -6.5084e-02],
         [-3.3123e-01, -3.9320e-02,  1.2404e-01,  6.3372e-01,  1.4701e+00],
         [-6.3385e-02, -3.6675e-01, -6.0017e-01,  2.7738e-01, -5.8430e-01],
         [ 5.4820e-01,  6.6289e-01, -9.0787e-01, -1.7304e+00, -1.6757e+00],
         [ 8.9007e-01,  8.9525e-02, -3.1659e-01, -6.3764e-02, -1.1951e-01],
         [ 2.1790e-01, -6.2005e-01,  7.9539e-01,  4.4432e-01,  6.2194e-01],
         [-8.6940e-01,  9.3615e-01, -5.5359e-01,  1.2746e+00, -1.9499e-01],
         [ 1.8527e-01, -4.0601e-01, -8.3505e-01, -1.1950e+00,  1.2917e+00],
         [ 6.6968e-01,  2.0280e-01, -1.4884e+00, -2.5294e-01,  3.8261e-01],
         [ 8.2815e-02,  1.1958e+00, -4.3021e-01, -1.1981e+00, -1.2425e-01],
         [-2.4096e-01, -3.4771e-01, -8.8776e-02, -9.5370e-01,  1.2759e+00],
         [ 1.0217e+00, -1.4085e+00, -6.8496e-01, -2.3703e-01,  2.9795e-01],
         [ 5.9004e-01,  6.4147e-02,  9.5276e-01,  3.9615e-01, -1.

In [35]:
def _collate_fn(batch, window_size: int, word_to_index: Dict[str, int]) -> any:
    X, y = zip(*batch)
    X = [pad_window(s, window_size=window_size) for s in X]
    X = [tokens_to_indices(s, word_to_index) for s in X]
    X = [torch.LongTensor(X_i) for X_i in X]

    pad_token_index = word_to_index['<pad>']
    X_padded = nn.utils.rnn.pad_sequence(X, batch_first=True, padding_value=pad_token_index)

    lengths = torch.LongTensor([len(label) for label in y])

    y = [torch.LongTensor(y_i) for y_i in y]
    y_padded = nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=0)

    return X_padded, y_padded, lengths

In [36]:
data = list(zip(train_sentences, train_labels))
collate_fn = partial(_collate_fn, window_size=2, word_to_index=word_to_index)

loader = DataLoader(data, batch_size=2, shuffle=True, collate_fn=collate_fn)

for counter, (batched_X, batched_y, batched_lengths) in enumerate(loader):
    print(f"Iteration {counter}")
    print("Batched Input:")
    print(batched_X)
    print("Batched Labels:")
    print(batched_y)
    print("Batched Lengths:")
    print(batched_lengths)
    print("")

Iteration 0
Batched Input:
tensor([[  0,   0, 112,  45, 122,  69,   6,  46,   4, 112,  91,   0,   0],
        [  0,   0, 112,  75, 101, 122,   2,  31,  60,  13,   0,   0,   0]])
Batched Labels:
tensor([[0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0]])
Batched Lengths:
tensor([9, 8])

Iteration 1
Batched Input:
tensor([[  0,   0, 112,   7, 123,  99,   6,  92,   0,   0,   0],
        [  0,   0,  52,  89, 131, 129,   2,  13,  87,   0,   0]])
Batched Labels:
tensor([[0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 0]])
Batched Lengths:
tensor([6, 7])

Iteration 2
Batched Input:
tensor([[  0,   0, 112,  72, 101, 122,   2, 105,  85,  88,   0,   0],
        [  0,   0, 112,  12,  49,  17,  14,  39,   0,   0,   0,   0]])
Batched Labels:
tensor([[0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 0]])
Batched Lengths:
tensor([8, 6])

Iteration 3
Batched Input:
tensor([[  0,   0,  54,  36, 123,   2, 106,  96,  77,  51,   0,   0,   0,   0],
        [  0,   0, 112, 110, 122,  

In [37]:
print('Original Tensor:')
print(batched_X)

chunk = batched_X.unfold(1, 2 * 2 + 1, 1)  # 2 * window_size + 1
print('\nWindows:')
print(chunk)

Original Tensor:
tensor([[  0,   0,  52,  16,   2,  83,  77,  19,  64, 100,   0,   0]])

Windows:
tensor([[[  0,   0,  52,  16,   2],
         [  0,  52,  16,   2,  83],
         [ 52,  16,   2,  83,  77],
         [ 16,   2,  83,  77,  19],
         [  2,  83,  77,  19,  64],
         [ 83,  77,  19,  64, 100],
         [ 77,  19,  64, 100,   0],
         [ 19,  64, 100,   0,   0]]])


In [38]:
class WordWindowClassifier(nn.Module):
    def __init__(self, hyperparameters, vocab_size, padding_idx=0):
        super(WordWindowClassifier, self).__init__()

        self.window_size = hyperparameters['window_size']
        self.embed_dim = hyperparameters['embed_dim']
        self.hidden_dim = hyperparameters['hidden_dim']
        self.freeze_embeddings = hyperparameters['freeze_embeddings']

        # Embedding Layer
        self.embeds = nn.Embedding(vocab_size, self.embed_dim, padding_idx=padding_idx)
        if self.freeze_embeddings: self.embed_layer.weight.requires_grad = False

        # Hidden Layer
        full_window_size = 2 * self.window_size + 1
        self.hidden_layer = nn.Sequential(
            nn.Linear(full_window_size * self.embed_dim, self.hidden_dim),
            nn.Tanh()
        )

        # Output layer
        self.output_layer = nn.Linear(self.hidden_dim, 1)

        # Probabilities
        self.probabilities = nn.Sigmoid()

    def forward(self, inputs):
        B, _ = inputs.size()

        # Reshaping
        token_windows = inputs.unfold(1, 2 * self.window_size + 1, 1)
        _, adjusted_length, _ = token_windows.size()
        assert token_windows.size() == (B, adjusted_length, 2 * self.window_size + 1)

        # Embedding
        embedded_windows = self.embeds(token_windows)

        # Reshaping
        embedded_windows = embedded_windows.view(B, adjusted_length, -1)

        # Layer 1
        layer_1 = self.hidden_layer(embedded_windows)

        # Layer 2
        output = self.output_layer(layer_1)

        # Softmax Score
        output = self.probabilities(output)
        output = output.view(B, -1)

        return output

## Training

In [39]:
data = list(zip(train_sentences, train_labels))
batch_size = 2
shuffle = True
window_size = 2
collate_fn = partial(_collate_fn, window_size=window_size, word_to_index=word_to_index)

loader = DataLoader(data, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)

model_hyperparameters = {
    'batch_size': 4,
    'window_size': 2,
    'embed_dim': 25,
    'hidden_dim': 25,
    'freeze_embeddings': False
}

vocab_size = len(word_to_index)
model = WordWindowClassifier(model_hyperparameters, vocab_size)

### Optimizer Definition

In [40]:
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Loss and Train Functions

In [41]:
def loss_function(batch_outputs, batch_labels, batch_lenghts) -> float:
    bceloss = nn.BCELoss()
    loss = bceloss(batch_outputs, batch_labels.float())
    
    # Rescaling
    loss = loss/batched_lengths.sum().float()

    return loss


def train_epoch(loss_function, optimizer, model, loader):
    total_loss = 0
    for batch_inputs, batch_labels, batch_lengths in loader:
        optimizer.zero_grad()  # clear gradient
        outputs = model.forward(batch_inputs)  # forward pass
        loss = loss_function(outputs, batch_labels, batch_lengths)  # batch loss
        loss.backward()  # gradient
        optimizer.step()
        total_loss += loss.item()
    return total_loss


def train(loss_function, optimizer, model, loader, num_epochs=10000):
    for epoch in range(0, num_epochs):
        epoch_loss = train_epoch(loss_function, optimizer, model, loader)
        if epoch % (int(num_epochs * 0.1)) == 0 or epoch == num_epochs-1: print(f'Epoch: {epoch+1} - Loss: {epoch_loss}')

### Training

In [42]:
num_epochs = 1000
train(loss_function, optimizer, model, loader, num_epochs=num_epochs)

Epoch: 1 - Loss: 0.8960276804864407
Epoch: 101 - Loss: 4.583937402458105e-05
Epoch: 201 - Loss: 2.019984798096175e-05
Epoch: 301 - Loss: 7.288666928673138e-06
Epoch: 401 - Loss: 2.3485473299444948e-06
Epoch: 501 - Loss: 5.880756521747799e-07
Epoch: 601 - Loss: 2.6456731649915355e-07
Epoch: 701 - Loss: 2.147103441707543e-07
Epoch: 801 - Loss: 8.981876137292844e-08
Epoch: 901 - Loss: 3.357754924104839e-08
Epoch: 1000 - Loss: 2.6768253710840462e-08


## Model's Predictions

In [45]:
test_corpus = ["The sky is orange and blue.",
               "When the sun goes down, the sky turns black.",
               "The car is purple!",
               "Is the car violet?",
               "No color in there.",
               "I like ham when it's pink."]
test_sentences = [tokenize(sentence) for sentence in test_corpus]
test_labels = [[0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 0, 0, 0, 0, 1],
               [0, 0, 0, 1],
               [0, 0, 0, 1],
               [0, 0, 0, 0],
               [0, 0, 0, 0, 0, 1]]

verbose = True
test_data = list(zip(test_sentences, test_labels))
batch_size = 1
shuffle = False
window_size = 2
collate_fn = partial(_collate_fn, window_size=window_size, word_to_index=word_to_index)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

for counter, (test_instance, labels, _) in enumerate(test_loader):
    print(f'Sample {counter+1}')
    outputs = model.forward(test_instance)[0]
    if verbose:
        print(f'   Labels: {labels}\n   Outputs: {outputs}')
    nb_colors = labels.sum()
    if nb_colors == 0:
        print('   No color to detect.\n')
        continue
    colors_indexes = torch.topk(outputs.flatten(), nb_colors).indices
    if nb_colors == 1:
        print(f'   Detected color: {index_to_word[int(test_instance[0, window_size+colors_indexes[0]])]}.\n')
    else:
        colors = ', '.join(index_to_word[int(test_instance[0, window_size+index])] for index in colors_indexes)
        print(f'   Detected colors: {colors}.\n')

Sample 1
   Labels: tensor([[0, 0, 0, 1, 0, 1]])
   Outputs: tensor([1.0378e-09, 9.4432e-10, 1.9697e-09, 9.9998e-01, 2.5041e-01, 1.0000e+00],
       grad_fn=<SelectBackward0>)
   Detected colors: blue, orange.

Sample 2
   Labels: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1]])
   Outputs: tensor([9.6650e-01, 1.1623e-09, 1.1613e-09, 6.8537e-04, 1.4523e-02, 9.5724e-10,
        1.0006e-09, 4.5953e-09, 1.0000e+00], grad_fn=<SelectBackward0>)
   Detected color: black.

Sample 3
   Labels: tensor([[0, 0, 0, 1]])
   Outputs: tensor([9.8084e-10, 9.3371e-10, 2.4507e-09, 9.9999e-01],
       grad_fn=<SelectBackward0>)
   Detected color: purple.

Sample 4
   Labels: tensor([[0, 0, 0, 1]])
   Outputs: tensor([1.7392e-08, 9.2324e-10, 5.7092e-09, 1.1634e-06],
       grad_fn=<SelectBackward0>)
   Detected color: <unk>.

Sample 5
   Labels: tensor([[0, 0, 0, 0]])
   Outputs: tensor([9.9834e-01, 6.2777e-02, 1.3445e-09, 5.8848e-07],
       grad_fn=<SelectBackward0>)
   No color to detect.

Sample 6
   Labels: ten

The performances of the models here are quite good yet it troubles on certain situations.\
To prevent from these, it could be relevant to increase the size of the dataset, look for a word-reduction method (stemming or lemmatizing) or optimize the structure of the neural network as it remains very basic.